In [1]:
#Import all the necessary libraries
import os
import numpy as np
import pandas as pd
from keras.layers import Dense,LSTM,SimpleRNN, Embedding
from keras.models import Sequential
from keras.utils.np_utils import to_categorical
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import seaborn as sns 

Using TensorFlow backend.


In [2]:
#setting working directory 
os.chdir("E:\\INSOFE\\text mining\\phone_data")

In [3]:
actual_data=pd.read_excel("Data-for-Classification-20180320 - V2.xlsx",header=0)

In [8]:
actual_data.columns

Index(['Summary (Input', 'DATA (Input)', 'categories (Output)',
       'sub_categories (Output)', 'previous_appointment (Output)'],
      dtype='object')

In [7]:
actual_data.drop('SLNO',inplace=True,axis=1)

In [9]:
#Changing the column names
col_names=["summary","data","categories","sub_categories","appointment"]
actual_data.columns=col_names

In [10]:
actual_data.columns

Index(['summary', 'data', 'categories', 'sub_categories', 'appointment'], dtype='object')

In [11]:
null_columns=actual_data.columns[actual_data.isnull().any()]
actual_data[null_columns].isnull().sum()

summary        3371
data           2706
appointment       2
dtype: int64

In [22]:
nulls = actual_data[actual_data.data.isnull()]
nulls.head(5)

,summary,data,categories,sub_categories,appointment
9,Phone Note,NaN,MISCELLANEOUS,OTHERS,No
22,ADD ON PPC AT 2:40PM,NaN,APPOINTMENTS,NEW APPOINTMENT,No
31,rov 5/5@11a w/ Liz,NaN,APPOINTMENTS,NEW APPOINTMENT,No
39,Same Day Appt Questions,NaN,APPOINTMENTS,QUERY ON CURRENT APPOINTMENT,No
73,Side effects on Lamictal,NaN,ASK_A_DOCTOR,MEDICATION RELATED,No


In [25]:
#to drop the NANs
actual_data.dropna(axis = 0,how = 'any',inplace=True)
actual_data = actual_data.reset_index(drop=True)

In [26]:
actual_data.head(10)

,summary,data,categories,sub_categories,appointment
0,Pt aware that he needs ROV for refill,Patient is returning nurse call. He is unable ...,PRESCRIPTION,REFILL,No
1,Mom wants to know if the Focalin needs some do...,school teacher is reporting pt is not able to ...,ASK_A_DOCTOR,MEDICATION RELATED,No
2,pt called to discuss nortryptiline. she says s...,Prescription Prescription Comments pt called ...,ASK_A_DOCTOR,MEDICATION RELATED,No
3,FYI Nortryptline medication.,Prescription Prescription Comments pt called ...,MISCELLANEOUS,OTHERS,No
4,Letter of patient establishment request,Requesting a letter stating that pt is indeed ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
5,Appt question,Appointments Pt mom wants to know if pt should...,APPOINTMENTS,QUERY ON CURRENT APPOINTMENT,No
6,dizzy & double vision past 45 mins after CT,had CT this a. m. in Smithfield and is feeling...,ASK_A_DOCTOR,SYMPTOMS,No
7,Please refax neurocog order to new fac; Wake ...,Dr. is too far booked out per pt and like the ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No
8,pt wants to reschedule Epidural from 04/30 ; p...,Appointments pt wants to reschedule Epidural f...,APPOINTMENTS,RESCHEDULING,No
9,rov,xxx Appointments Pt calling to reschedule appt...,APPOINTMENTS,NEW APPOINTMENT,No


In [41]:
#Converting datatype of summary feature
actual_data.summary =actual_data.summary.astype('str')
#Converting datatype of data feature
actual_data.data =actual_data.data.astype('str')
#Converting datatype of category feature
actual_data.categories=actual_data.categories.astype("category")
#Converting datatype of sub_categories feature
actual_data.sub_categories=actual_data.sub_categories.astype("category")
#Converting datatype of appointment feature
actual_data.appointment=actual_data.appointment.astype("category")

In [42]:
#Converting to lower case
actual_data['summary'] = actual_data['summary'].astype(str).str.lower()
actual_data["data"] = actual_data["data"].astype(str).str.lower()
actual_data["categories"] = actual_data["categories"].map(lambda x: x.upper())
actual_data["sub_categories"] = actual_data["sub_categories"].map(lambda x: x.upper())
actual_data["appointment"] = actual_data["appointment"].map(lambda x: x.upper())

In [46]:
actual_data.categories.value_counts()

PRESCRIPTION     14442
ASK_A_DOCTOR     11670
APPOINTMENTS     10924
MISCELLANEOUS     9943
LAB               4220
JUNK                 3
Name: categories, dtype: int64

In [47]:
actual_data.dtypes

summary             object
data                object
categories        category
sub_categories    category
appointment       category
dtype: object

In [51]:
actual_data.data[0:3]

0    patient is returning nurse call. he is unable ...
1    school teacher is reporting pt is not able to ...
2    prescription  prescription comments pt called ...
Name: data, dtype: object

In [52]:
#Some Preprocessing
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def raw_to_prepwithtokenize( raw_review ):
    review_text = BeautifulSoup(raw_review).get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    low_case = letters_only.lower()
    words = nltk.word_tokenize(low_case)
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    processed_sentence = " ".join(meaningful_words)
    return(processed_sentence)

In [53]:
num_reviews=len(actual_data)
num_reviews

51202

In [54]:
process=[]
for i in range (0,num_reviews):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    process.append(raw_to_prepwithtokenize(actual_data['data'][i]))

no.of rows reviews processed: 0


E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000


E:\Anaconda3\lib\site-packages\bs4\__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


no.of rows reviews processed: 50000


In [55]:
process_summary=[]
for i in range (0,num_reviews):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    process_summary.append(raw_to_prepwithtokenize(actual_data['summary'][i]))

no.of rows reviews processed: 0


E:\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file E:\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000


In [60]:
process[0:3]

['patient returning nurse call unable make appt without talking fin service dept however needs medication worried issue without medication please call patient discuss pt returned phone call please call back advise problem without insurance instructions come appt refills without one david rn spoke pt relayed requested speak financial services rn transferred business office rn requested business office call matter completed hollie saltis rn ok david',
 'school teacher reporting pt able sit still mom wants know focalin needs dosage adjusting something pt could take school staff could administer please call back discuss mom sts patient alot issues meds effectiveness lov rescheduled show mom apologized show sts disconnected home number use cell appt scheduled advd phone call completed appt scheduled marcia richardson lpn',
 'prescription prescription comments pt called discuss nortryptiline says weird taste mouth pt also sweats heavily please call discuss rna pt states food favor last week 

In [61]:
process_summary[0:3]

['pt aware needs rov refill',
 'mom wants know focalin needs dosage adjusting',
 'pt called discuss nortryptiline says weird tas']

In [171]:
#doing TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features= 100)

In [172]:
#TF-IDF for data
tfidf_data = tfidf_vectorizer.fit_transform(process)


In [173]:
dense_data = tfidf_data.todense()

In [174]:
dense_data.shape

(51202, 100)

In [175]:
tfidf_vectorizer_summary = TfidfVectorizer(ngram_range=(1,2),max_features= 30)

In [176]:
#TF-IDF for summary
tfidf_summary = tfidf_vectorizer_summary.fit_transform(process_summary)

In [177]:
dense_summary = tfidf_summary.todense()

In [178]:
dense_summary.shape

(51202, 30)

In [179]:
feature_names = tfidf_vectorizer.get_feature_names()
print(len(feature_names))
print(feature_names[:50])

100
['advise', 'also', 'appointments', 'appt', 'back', 'bid', 'call', 'call back', 'call completed', 'called', 'caller', 'caller verbalized', 'completed', 'days', 'details', 'discuss', 'dr', 'fax', 'faxed', 'fill', 'get', 'give', 'instructions', 'know', 'last', 'like', 'lisa', 'lpn', 'may', 'md', 'medfusion', 'medication', 'medications', 'medications updated', 'message', 'mg', 'mg tabs', 'mom', 'mri', 'nc', 'need', 'needs', 'new', 'note', 'notified', 'office', 'one', 'oral', 'order', 'pain']


In [90]:
#import pandas as pd
#x = pd.DataFrame(dense_data)
#x.columns = tfidf_vectorizer.get_feature_names()
#x['text'] = process
#x.to_csv('mytfidf.csv', index = False)
#x

In [181]:
dense_data[0]

matrix([[ 0.13512087,  0.        ,  0.        ,  0.26637162,  0.12610119,
          0.        ,  0.41132895,  0.15350416,  0.        ,  0.        ,
          0.        ,  0.        ,  0.12871165,  0.        ,  0.        ,
          0.16994023,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.1618985 ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.28147335,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.15443802,  0.        ,  0.        ,  0.        ,
          0.34191045,  0.19028484,  0.        ,  0.        ,  0.        ,
          0.20188409,  0.        ,  0.        ,  0.        ,  0.12154242,
          0.13066822,  0.1996436 ,  0.        ,  0.28165184,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.

In [182]:
df_data = pd.DataFrame(dense_data)
df_summary = pd.DataFrame(dense_summary)

In [183]:
print(df_data.shape)
print(df_summary.shape)

(51202, 100)
(51202, 30)


In [184]:
col_names =[]
for i in range (100,130):
    col_names.append(i)

In [185]:
df_summary.columns=col_names

In [186]:
preprocessed_data=pd.concat([df_data,df_summary],axis=1)

In [187]:
preprocessed_data.shape

(51202, 130)

In [188]:
x_train = preprocessed_data

In [189]:
y_train = actual_data.categories

In [190]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [191]:
from sklearn.ensemble import RandomForestClassifier
model_random = RandomForestClassifier(n_estimators = 500)
model_fit = model_random.fit(X_train,Y_train)

In [192]:
pred_test = model_fit.predict(X_test)

In [193]:
from sklearn.metrics import accuracy_score,classification_report,precision_score,recall_score
print(accuracy_score(Y_test,pred_test))

0.76242554438


In [194]:
pred_train = model_fit.predict(X_train)

In [195]:
accuracy_score(Y_train,pred_train)

0.99216327726373865

In [163]:
unique_labels = list(actual_data.categories.unique())
train_y = np.array([unique_labels.index(i) for i in Y_train])
train_y = to_categorical(train_y)
test_y = np.array([unique_labels.index(i) for i in Y_test])
test_y = to_categorical(test_y)

In [164]:
print(train_y.shape)
print(test_y.shape)

(40961, 6)
(10241, 6)


In [165]:
import keras
from keras.layers import Dense
from keras.models import Sequential

In [166]:
model = Sequential()
model.add(Dense(units= 10,kernel_initializer='uniform',input_dim = 1100,activation='relu'))
model.add(Dense(units = 10,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units = 6,kernel_initializer= 'uniform',activation='sigmoid'))

In [167]:
model.compile(optimizer='adam',loss = 'categorical_crossentropy',metrics=['accuracy'])

In [168]:
model.fit(X_train,train_y,batch_size=10,epochs = 10)

Epoch 1/10


KeyError: '[27259  6641 38626  7131 23638 31195 38124 40316 20301 14754] not in index'